# image converter

- 將 MVTect 的 dataset 資料夾中所有圖片轉換成 256x256
- 生成資料的時候連同 augmentation 的資料也一同生成

In [1]:
import random
from PIL import Image
import os

import numpy as np
import skimage.io as io
from skimage.transform import rotate, AffineTransform, warp, resize
from skimage.util import random_noise

from skimage.filters import gaussian
import matplotlib.pyplot as plt

In [2]:
# TYPES = ["bottle", "cable", "capsule", "carpet", "grid", "hazelnut", "leather", "metal_nut", "pill", "screw", "tile", "toothbrush", "transistor", "wood", "zipper"]
TYPES = ["bottle"]
ROOT = 'MVTec/'
SIZE = (256, 256)
MESSUP = True

In [3]:
def saveResizeImage(im, savePath):
    image_resized = resize(im, SIZE, anti_aliasing=True)
    io.imsave(fname=savePath, arr=image_resized)

def saveRotateImage(im, savePath):
    angles = [90, 180, 270]
    image_resized = resize(im, SIZE, anti_aliasing=True)
    
    for angle in angles:
        image_rotated = rotate(image_resized, angle=angle)
        io.imsave(fname=savePath[:-4]+'_'+str(angle)+'.png', arr=image_rotated)

def saveFlipImage(im, savePath):
    image_resized = resize(im, SIZE, anti_aliasing=True)
    
    image_flipLR = np.fliplr(image_resized)
    image_flipUD = np.flipud(image_resized)
    
    io.imsave(fname=savePath[:-4]+'_LR.png', arr=image_flipLR)
    io.imsave(fname=savePath[:-4]+'_UD.png', arr=image_flipUD)

def saveNoiseImage(im, savePath, sigma=0.155):
    image_resized = resize(im, SIZE, anti_aliasing=True)
    
    image_random_noise = random_noise(image_resized, var=sigma**2)
    io.imsave(fname=savePath, arr=image_random_noise)

def saveBlueImage(im, savePath, sigma=1.5):
    image_resized = resize(im, SIZE, anti_aliasing=True)
    
    image_blurred = gaussian(image_resized, sigma=sigma, multichannel=True)
    io.imsave(fname=savePath, arr=image_blurred)

In [4]:
# create training set

for _type in TYPES:
    trainDataPath = ROOT+_type+'/train/good/'
    trainResizeDataPath = ROOT+_type+'/train_resize/train/'
    valResizeDataPath = ROOT+_type+'/train_resize/validation/'
    
    trainImages = os.listdir(trainDataPath)
    trainImages = [img for img in trainImages if img.endswith('.png')]
    random.shuffle(trainImages) if MESSUP else trainImages
    
    for index, imageName in enumerate(trainImages):
        image = io.imread(trainDataPath + imageName)

        # CheckDir
        if not os.path.isdir(trainResizeDataPath):
            os.makedirs(trainResizeDataPath)
            os.makedirs(ROOT+_type+'/train_resize/validation')
            
        
        savePath = valResizeDataPath if index < len(trainImages) // 10 else trainResizeDataPath
        
        # Save Image
        saveResizeImage(image, savePath+imageName)
#         saveRotateImage(image, savePath+imageName[:-4]+'_rotated.png')
#         saveFlipImage(image, savePath+imageName[:-4]+'_flipped.png')
#         saveNoiseImage(image, savePath+imageName[:-4]+'_noised.png')
#         saveBlueImage(image, savePath+imageName[:-4]+'_blurred.png')

/opt/conda/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/opt/conda/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [ ]:
# create testing set
for _type in TYPES:
    testDataPath = ROOT+_type+'/test/'
    testResizeDataPath = ROOT+_type+'/test_resize/'
    
    gtDataPath = ROOT+_type+'/ground_truth/'
    gtResizedDataPath = ROOT+_type+'/ground_truth_resize/'
    
    testDataType = os.listdir(testDataPath)
    
    if not os.path.isdir(testResizeDataPath):
        os.makedirs(testResizeDataPath)
        os.makedirs(testResizeDataPath+'all')
    if not os.path.isdir(gtResizedDataPath):
        os.makedirs(gtResizedDataPath)
        os.makedirs(gtResizedDataPath+'all')
        
    
    index = 0
    for anomalyType in testDataType:
        imagePath = testDataPath+anomalyType+'/' 
        savePath = testResizeDataPath+anomalyType+'/'
        
        gtPath = gtDataPath+anomalyType+'/' 
        gtSavePath = gtResizedDataPath+anomalyType+'/'
        
        if not os.path.isdir(savePath):
            os.makedirs(savePath)
        if not os.path.isdir(gtSavePath):
            os.makedirs(gtSavePath)
        
        testImages = os.listdir(imagePath)
        for imageName in testImages:
            if imageName.endswith('.png'):
                im = Image.open(imagePath+imageName)
                im = im.resize(SIZE)
                im.save(savePath + imageName)
                
                if anomalyType != 'good':
                    im.save(testResizeDataPath+'all/'+str(index)+'.png')
                    _gt = Image.open(gtPath+imageName[:-4]+'_mask.png')
                    _gt = _gt.resize(SIZE)
                    _gt.save(gtSavePath+imageName[:-4]+'_mask.png')
                    _gt.save(gtResizedDataPath+'all/'+str(index)+'.png')
                
                index+=1
            else:
                print(imagePath+imageName, "is not a picture")

In [6]:
import random
a = [1, 2, 3, 0, 1, 2, 0, 54,3, 3, 123,123]
random.shuffle(a)
print(a)

[1, 3, 3, 123, 1, 3, 123, 0, 2, 54, 0, 2]


# Mask generater

1. Test 用的，用來生成圖片的 mask

In [31]:
import numpy
import matplotlib.pyplot as plt
from PIL import Image
import torch

In [32]:
# 參數設定
image_size = (256, 256)
mask_size = (64, 64)

In [33]:
bg = numpy.uint8(numpy.zeros(image_size))
m = numpy.uint8(255*numpy.ones(mask_size))

divisor = numpy.uint8(255*numpy.zeros(image_size))
x = 0
y = 0
count = 0
while x <= 192 and y <= 192:
    background = Image.fromarray(bg)
    mask = Image.fromarray(m)

    background.paste(mask, (x, y))
    divisor = divisor + (numpy.array(background.getdata()).reshape(background.size[0], background.size[1])/255).astype(numpy.int8)
    #background.save('MVTec/mask/' + str(count) +'.png')
    x += 32
    if x == 224:
        x = 0
        y += 32
    count += 1
    
divisor = numpy.reshape(divisor, (1,1,256,256))
tmp = divisor
divisor = numpy.concatenate((divisor, tmp), axis=1)
divisor = numpy.concatenate((divisor, tmp), axis=1)
divisor = torch.from_numpy(divisor)
#Image.fromarray(divisor,'RGB').save('MVTec/mask/divisor.png')

In [8]:
'''bg = numpy.uint8(numpy.zeros(mask_size))
m = numpy.uint8(255*numpy.ones(mask_size))
background = Image.fromarray(bg)
mask = Image.fromarray(m)
test = background + mask + background + mask
test.save('MVTec/mask/test.png')'''

TypeError: unsupported operand type(s) for +: 'Image' and 'Image'

# Test image generater
1. 生成 inpaint 後的圖片
2. 會將原始圖片切成 16 等分，每一等分都丟進去 netG 進行 inpaint，之後 16 快圖片拼起來

In [34]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import torchvision.utils as vutils

from model.networks import Generator
from utils.tools import get_config, is_image_file, default_loader, normalize, get_model_list

from tqdm import tqdm

In [35]:
config_path = "configs/config.yaml"
seed = random.randint(1, 10000)
root = "MVTec/grid/test_resize/all/" #capsule
# root = "MVTec/grid/train_resize/"
maskRoot = "MVTec/mask2/"
outputPath = "output/"
iteration=0 #85000

In [36]:
config = get_config(config_path)

# CUDA configuration
cuda = config['cuda']
device_ids = config['gpu_ids']
if cuda:
    os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(i) for i in device_ids)
    device_ids = list(range(len(device_ids)))
    config['gpu_ids'] = device_ids
    cudnn.benchmark = True

# Set random seed
print("Random seed: {}".format(seed))
random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed_all(seed)

print("Configuration: {}".format(config))

# 生成的部分
# 1. 從 checkpoint 裡面找到 weight 載入 Generator
with torch.no_grad():
    # Read all image name from root
    imageNames = os.listdir(root)

    for name in imageNames:
        if is_image_file(root + name):
            inpainted_image = torch.zeros(1,3,256,256)
            error_counts = torch.zeros(1,3,256,256)
            maskNames = os.listdir(maskRoot)
            for maskName in maskNames:
                if maskName=='.ipynb_checkpoints':
                    continue
                # Test a single masked image with a given mask
                origin_img = default_loader(root+name)
                x = default_loader(root+name)
                mask = default_loader(maskRoot+maskName)
                
                x = transforms.Resize(config['image_shape'][:-1])(x)
                x = transforms.CenterCrop(config['image_shape'][:-1])(x)
                mask = transforms.Resize(config['image_shape'][:-1])(mask)
                mask = transforms.CenterCrop(config['image_shape'][:-1])(mask)
                origin_img = transforms.Resize(config['image_shape'][:-1])(origin_img)
                origin_img = transforms.CenterCrop(config['image_shape'][:-1])(origin_img)

                x = transforms.ToTensor()(x)
                origin_img = transforms.ToTensor()(origin_img)
                mask = transforms.ToTensor()(mask)[0].unsqueeze(dim=0)


                x = normalize(x)
                x = x * (1. - mask)
                x = x.unsqueeze(dim=0)

                origin_img = normalize(origin_img)
                origin_img = origin_img.unsqueeze(dim=0)

                mask = mask.unsqueeze(dim=0)

                # Set checkpoint path
                checkpoint_path = os.path.join('checkpoints', config['dataset_name'], config['mask_type'] + '_' + config['expname'])

                # Define the trainer
                netG = Generator(config['netG'], cuda, device_ids)
                netG = netG.cuda()
                # Resume weight
                last_model_name = get_model_list(checkpoint_path, "gen", iteration=iteration)
                netG.load_state_dict(torch.load(last_model_name))
                model_iteration = int(last_model_name[-11:-3])

                if cuda:
                    netG = nn.parallel.DataParallel(netG, device_ids=device_ids)
                    x = x.cuda()
                    mask = mask.cuda()
                    origin_img = origin_img.cuda()

                # Inference
                x1, x2, offset_flow = netG(x, mask)

                inpainted_result = x2 * mask + x
                inpainted_patten = x2 * mask
                
                crop_origin_img = origin_img * mask
                
                error = torch.abs(crop_origin_img - inpainted_patten)
                inpainted_image += inpainted_patten.cpu() 
                error_counts += error.cpu()
            #vutils.save_image(inpainted_image, outputPath+name[:-4]+'_inpainted_nodivisor.png', padding=0, normalize=True)
            inpainted_image = torch.div(inpainted_image.float(),divisor.float())
            error_counts = torch.div(error_counts.float(),divisor.float())
            vutils.save_image(inpainted_image, outputPath+name[:-4]+'_inpainted.png', padding=0, normalize=True)
            vutils.save_image(origin_img, outputPath+name, padding=0, normalize=True)
            vutils.save_image(error_counts, outputPath+name[:-4]+'_error.png', padding=0, normalize=True)
            print("Saved the inpainted result to {}".format(outputPath+name))
        else:
            print("{} is not an image file.".format(root+name))

Random seed: 4650
Configuration: {'dataset_name': 'MVTec/grid_withPerceptualSimilarity_20200715/', 'data_with_subfolder': False, 'train_data_path': 'MVTec/grid/train_resize', 'val_data_path': None, 'resume': None, 'batch_size': 16, 'image_shape': [256, 256, 3], 'mask_shape': [64, 64], 'mask_batch_same': True, 'max_delta_shape': [32, 32], 'margin': [0, 0], 'discounted_mask': True, 'spatial_discounting_gamma': 0.9, 'random_crop': True, 'mask_type': 'hole', 'mosaic_unit_size': 12, 'expname': 'benchmark', 'cuda': True, 'gpu_ids': [0], 'num_workers': 32, 'lr': 0.0001, 'd_lr': 1e-05, 'beta1': 0.5, 'beta2': 0.9, 'n_critic': 5, 'niter': 500000, 'print_iter': 100, 'viz_iter': 1000, 'viz_max_out': 16, 'snapshot_save_iter': 5000, 'coarse_l1_alpha': 1.2, 'l1_loss_alpha': 1.2, 'ae_loss_alpha': 1.2, 'myd_loss_alpha': 0.5, 'global_wgan_loss_alpha': 1.0, 'gan_loss_alpha': 0.001, 'wgan_gp_lambda': 10, 'netG': {'input_dim': 3, 'ngf': 32}, 'netD': {'input_dim': 3, 'ndf': 64}}
Saved the inpainted result t

KeyboardInterrupt: 

### 